<a href="https://colab.research.google.com/github/Ignvz/copert_5/blob/main/Factores_emision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#If used from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd "/content/drive/MyDrive/Factores de emision"
!ls

/content/drive/.shortcut-targets-by-id/1rXzOeDHO59D46y_UeQ5BIKirkwBHax7H/Factores_de_emision
'1.A.3.b.i-iv Road Transport Appendix 4 Emission Factors_2020Sept.xlsx'
'Actividad Vehicular.csv'
 BC_emissions.xlsx
 BC_factors.csv
 C5_reducido_prueba.xlsx
 C5_reducido.xlsx
 CH4_emissions.xlsx
 CH4_factors.csv
 CH4_reducido.xlsx
 CO2_emissions.xlsx
 CO2_factors.csv
 CO2_factors.gsheet
 CO2_reducido.xlsx
 CO_emissions.xlsx
 CO_factors.csv
 Copert5.xlsx
 CO_reducido.xlsx
 Euro_7.xlsx
 NOx_emissions.xlsx
 NOx_factors.csv
 NOx_reducido.xlsx
 PM_emissions.xlsx
 PM_factors.csv
 PM_reducido.xlsx
 SOx_emissions.xlsx
 SOx_factors.csv
 Velocidades.csv
 VOC_emissions.xlsx
 VOC_factors.csv
 VOC_reducido.xlsx


In [523]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

In [493]:
def copert5(alpha, beta, gamma, delta, epsilon, zeta, hta, vm, redFactor):
  if epsilon*vm**2+zeta*vm+hta == 0:
    return 0
  else:
    return (1-redFactor)*(alpha*vm**2 + beta*vm + gamma + delta/vm)/(epsilon * vm**2 + zeta*vm+ hta)

In [524]:
#cargando. excels
xls = pd.ExcelFile('Copert5.xlsx')
df = pd.read_excel(xls,'HOT_EMISSIONS_PARAMETERS', index_col=None)

In [525]:
#Filtrando usando REGEX a las categorías vehiculares a utilizar
df = df.loc[
      ((df['Fuel'] == 'Petrol') |  (df['Fuel'] == 'Diesel')) &
      df.Segment.str.contains('^Medium') | #VLP, TAX
      df.Segment.str.contains("^N1-II$")  | #VLC
      df.Segment.str.contains('^Art*|^Rigid 28 *|^Rigid >32*') | #CAM PES RIGID>28 t and articulated
      df.Segment.str.contains('^Rigid 14 *|^Rigid 20 *|^Rigid 26 *') |#CAM MED RIGID 14-28 t
      df.Segment.str.contains('^Rigid <=7,5 *|^Rigid 7,5 -*|^Rigid 12 *') |#CAM LIV
      df.Segment.str.contains('^Urban Buses Standard 15 *') | #Bus RIG
      df.Segment.str.contains('^Urban Buses Articulated >18 t') | #BUS ART
      df.Segment.str.contains('^Coaches Standard <=18 t') | #BUS INT PROV
      df.Segment.str.contains("^Motorcycles 2-stroke >50 *") #MOT
]


In [526]:
#Cambiando a la clasificacion a utilizar
df.loc[df['Segment'] == 'Medium', 'Segment'] = 'VLP'
df.loc[df['Segment'] == 'N1-II', 'Segment'] = 'VLC'
df.loc[df.Segment.str.contains('^Rigid <=7,5 *|^Rigid 7,5 -*|^Rigid 12 *'), 'Segment'] = 'CAM LIV'
df.loc[df.Segment.str.contains('^Rigid 14 *|^Rigid 20 *|^Rigid 26 *'), 'Segment'] = 'CAM MED'
df.loc[df.Segment.str.contains('^Art*|^Rigid 28 *|^Rigid >32*'), 'Segment'] = 'CAM PES'
df.loc[df.Segment.str.contains('^Urban Buses Standard 15 *'), 'Segment'] = 'BUS RIG'
df.loc[df.Segment.str.contains('^Urban Buses Articulated >18 t'), 'Segment'] = 'BUS ART'
df.loc[df.Segment.str.contains('^Coaches Standard <=18 t'), 'Segment'] = 'BUS INT PROV'
df.loc[df.Segment.str.contains("^Motorcycles 2-stroke >50 *"), 'Segment'] = 'MOT'
#df.to_excel('C5_reducido.xlsx',index = False)
#df.loc[df.Segment.str.contains('^CAM MED*')] 

In [527]:
#df = pd.read_excel('C5_reducido_prueba.xlsx')
df = df.loc[ ((df['Road Slope'].isnull()) & (df['Load'].isnull())) | #todas las pendientes y las que no tienen pendiente
       ((df['Load'] == 0.5))] #carga 50%
#df.to_excel('C5_reducido_prueba.xlsx',index = False)
pd.unique(df["Road Slope"])

array([  nan, -0.06, -0.04, -0.02,  0.  ,  0.02,  0.04,  0.06])

In [528]:
#Seteando velocidad
velMin = 25
velMax = 80
df["velMin"] = velMin
df["velMax"] = velMax
df["velMax"] = np.where(df["Max Speed [km/h]"] < velMax, #mayor que el valor max
                       df["Max Speed [km/h]"], velMax)
df["velMin"] = np.where(df["Min Speed [km/h]"] > velMin, #menor que el valor min
                       df["Min Speed [km/h]"], velMin)

In [529]:
#Cambiando clasificaciones
df.loc[df['Euro Standard'] == 'PRE ECE', 'Euro Standard'] = 'No cat'
df.loc[df['Euro Standard'] == 'Euro 1', 'Euro Standard'] = 'E1'
df.loc[df['Euro Standard'] == 'Euro 2', 'Euro Standard'] = 'E2'
df.loc[df['Euro Standard'] == 'Euro 3', 'Euro Standard'] = 'E3'
df.loc[df['Euro Standard'] == 'Euro 4', 'Euro Standard'] = 'E4'
df.loc[df['Euro Standard'] == 'Euro 5', 'Euro Standard'] = 'E5'
df.loc[df['Euro Standard'] == 'Euro 6 a/b/c', 'Euro Standard'] = 'E6abc'
df.loc[df['Euro Standard'] == 'Euro 6 d', 'Euro Standard'] = 'E6d'
df.loc[df['Euro Standard'] == 'Conventional', 'Euro Standard'] = 'No cat'
df.loc[df['Euro Standard'] == 'Euro I', 'Euro Standard'] = 'E1'
df.loc[df['Euro Standard'] == 'Euro II', 'Euro Standard'] = 'E2'
df.loc[df['Euro Standard'] == 'Euro III', 'Euro Standard'] = 'E3'
df.loc[df['Euro Standard'] == 'Euro IV', 'Euro Standard'] = 'E4'
df.loc[df['Euro Standard'] == 'Euro V', 'Euro Standard'] = 'E5'
df.loc[df['Euro Standard'] == 'Euro VI A/B/C', 'Euro Standard'] = 'E6abc' #al parecer no hay diferencia en los parámetros estudiados
df.loc[df['Euro Standard'] == 'Euro VI D/E', 'Euro Standard'] = 'E6d'
df.loc[df['Fuel'] == 'Petrol', 'Fuel'] = 'Gasolina'
#df.to_excel('C5_reducido_prueba.xlsx',index = False)


In [530]:
df = df.loc[
            #Todo lo en blanco
            (df['Technology'].isnull()) |
            #Passenger Cars
            ((df['Category'] == 'Passenger Cars') & 
             (df['Technology'] == 'PFI') | (df['Technology'] == 'DPF')) |
            #Light Commercial Vehicles
            ((df['Category'] == 'Light Commercial Vehicles') & 
             (df['Technology'] == 'PFI') | (df['Technology'] == 'DPF')) |
             #Buses
            ((df['Category'] == 'Buses') & 
             (df['Technology'] == 'EGR') | (df['Technology'] == 'DPF+SCR')) |
            #Heavy Duty Trucks
            ((df['Category'] == 'Heavy Duty Trucks') & 
             (df['Technology'] == 'EGR') | (df['Technology'] == 'DPF+SCR'))
            ]

In [531]:
alpha = df[["Alpha"]].astype("float")
beta = df[["Beta"]].astype("float")
gamma = df[["Gamma"]].astype("float")
delta = df[["Delta"]].astype("float")
epsilon = df[["Epsilon"]].astype("float")
zeta = df[["Zita"]].astype("float")
hta = df[["Hta"]].astype("float")
vmUrb = df[["velMin"]].astype("float")
vmInt = df[["velMax"]].astype("float")
redFactor = df[["Reduction Factor [%]"]].astype("float")
#copert5(alpha, beta, gamma, delta, epsilon, zeta, hta, vm, redFactor)
discriminante = (epsilon.values*vmUrb.values**2+zeta.values*vmUrb.values+
                 hta.values)
df["EF[g/km]_ECF[MJ/km]_Urb"] = np.where(discriminante < 0, 0, (1 - redFactor.values)*((alpha.values * vmUrb.values**2 + beta.values * vmUrb.values 
+ gamma.values + (delta.values/vmUrb.values)) / (discriminante)))
discriminante = (epsilon.values*vmInt.values**2+zeta.values*vmInt.values+
                 hta.values)
df["EF[g/km]_ECF[MJ/km]_Int"] = np.where(discriminante < 0, 0, (1 - redFactor.values)*((alpha.values * vmInt.values**2 + beta.values * vmInt.values 
+ gamma.values + (delta.values/vmInt.values)) / (discriminante)))

In [532]:
cols = list(df.columns)
#df = df[cols[0:9]+cols[11:19]]
dfCO = df.loc[df['Pollutant'] == 'CO']
dfNOx = df.loc[df['Pollutant'] == 'NOx']
dfVOC = df.loc[df['Pollutant'] == 'VOC']
dfPM = df.loc[df['Pollutant'] == 'PM Exhaust']
dfCH4 = df.loc[df['Pollutant'] == 'CH4']
dfCO2 = df.loc[df['Pollutant'] == 'EC']

In [533]:
#Calculo CO2 y SOx
gasDensidad = 0.730 #Ton/m3
gasPcalor = 11200 #Kcal/kg 
dieselPcalor = 10900 #Kcal/kg 
dieselDensidad = 0.840 #ton/m3
tJouleAtCal = 2.39*10**(-1) #de joule a cal 
kCalLt = (gasDensidad*1000*gasPcalor*0.001)
kCalLt = (dieselDensidad*1000*dieselPcalor*0.001)
######## Fin Datos ##########
####### Urbano #######

FCgas_urb = (1000*dfCO2[["EF[g/km]_ECF[MJ/km]_Urb"]]*tJouleAtCal)/(gasPcalor/1000) #gr/km
FCdies_urb = (1000*dfCO2[["EF[g/km]_ECF[MJ/km]_Urb"]]*tJouleAtCal)/(dieselPcalor/1000) #gr/km
FC_urb = np.where(dfCO2[["Fuel"]]=="Gasolina", FCgas_urb, FCdies_urb) #diferenciando diesel y gasolina gr/km
co2Calc_urb = 44.0011*(FC_urb/(12.011+1.008*2))
soxCalc_urb = 2*5*(10**-5)*FC_urb
dfCO2[["km/lt_urb"]] = np.where(dfCO2[["Fuel"]]=="Gasolina", (FC_urb/(gasDensidad*10**3))**-1,(FC_urb/(dieselDensidad*10**3))**-1)
dfCO2[["CO2 g/km_urb"]] = co2Calc_urb
dfCO2[["SOx g/km_urb"]] = soxCalc_urb
#### Interurbano ####
FCgas_int = (1000*dfCO2[["EF[g/km]_ECF[MJ/km]_Int"]]*tJouleAtCal)/(gasPcalor/1000) #gr/km
FCdies_int = (1000*dfCO2[["EF[g/km]_ECF[MJ/km]_Int"]]*tJouleAtCal)/(dieselPcalor/1000) #gr/km
FC_int = np.where(dfCO2[["Fuel"]]=="Gasolina", FCgas_int, FCdies_int) #diferenciando diesel y gasolina gr/km
co2Calc_int = 44.0011*(FC_int/(12.011+1.008*2))
soxCalc_int = 2*5*(10**-5)*FC_int
dfCO2[["km/lt_int"]] = np.where(dfCO2[["Fuel"]]=="Gasolina", (FC_int/(gasDensidad*10**3))**-1,(FC_int/(dieselDensidad*10**3))**-1)
dfCO2[["CO2 g/km_int"]] = co2Calc_int
dfCO2[["SOx g/km_int"]] = soxCalc_int



<ipython-input-533-f239e16f6413>:17: RuntimeWarning: divide by zero encountered in reciprocal
  dfCO2[["km/lt_urb"]] = np.where(dfCO2[["Fuel"]]=="Gasolina", (FC_urb/(gasDensidad*10**3))**-1,(FC_urb/(dieselDensidad*10**3))**-1)
<ipython-input-533-f239e16f6413>:26: RuntimeWarning: divide by zero encountered in reciprocal
  dfCO2[["km/lt_int"]] = np.where(dfCO2[["Fuel"]]=="Gasolina", (FC_int/(gasDensidad*10**3))**-1,(FC_int/(dieselDensidad*10**3))**-1)


In [534]:
columns_text = ["Region", "Ambito", "Modo", "Motorizacion", "Norma", "Gas", "Unidad"]

In [521]:
#Una vez reducida a lo que se utilizara, se separan dataframes diferentes por contaminante.
#df = pd.read_excel('C5_reducido_prueba.xlsx')
dfCO.to_excel('CO_reducido.xlsx',index = False)
dfNOx.to_excel('NOx_reducido.xlsx',index = False)
dfVOC.to_excel('VOC_reducido.xlsx',index = False)
dfPM.to_excel('PM_reducido.xlsx',index = False)
dfCH4.to_excel('CH4_reducido.xlsx',index = False)
dfCO2.to_excel('CO2_reducido.xlsx',index = False)

In [522]:
gk = df.groupby(["Segment", "Pollutant", "Fuel", 'Euro Standard']).mean()
gk[["EF[g/km]_ECF[MJ/km]_Urb","EF[g/km]_ECF[MJ/km]_Int", "velMax", "velMin"]].to_excel("Resumen_categoria_combustible.xlsx",index = True)
df.to_excel('C5_reducido_prueba.xlsx',index = False)
#df2 = pd.read_excel('PM_reducido.xlsx')
gk2 = dfCO2.groupby(["Segment", "Pollutant", "Fuel", 'Euro Standard']).mean()
gk2.to_excel("ResumenCO2.xlsx", index=True)
gk2


Road Slope  Load  \
Segment Pollutant Fuel     Euro Standard                             
BUS ART EC        Diesel   E1                            0.0   0.5   
                           E2                            0.0   0.5   
                           E3                            0.0   0.5   
                           E4                            0.0   0.5   
                           E5                            0.0   0.5   
...                                                      ...   ...   
VLP     EC        Gasolina ECE 15/04                     NaN   NaN   
                           Euro 6 d-temp                 NaN   NaN   
                           Improved Conventional         NaN   NaN   
                           No cat                        NaN   NaN   
                           Open Loop                     NaN   NaN   

                                                  Min Speed [km/h]  \
Segment Pollutant Fuel     Euro Standard                             
BUS ART EC        Diesel   E1                            10.857143   
                           E2                            10.857143   
                           E3                            11.000000   
                           E4                            10.857143   
                           E5                             5.000000   
...                                                            ...   
VLP     EC        Gasolina ECE 15/04                     10.000000   
                           Euro 6 d-temp                  5.000000   
                           Improved Conventional         10.000000   
                           No cat                        10.000000   
                           Open Loop                     10.000000   

                                                  Max Speed [km/h]     Alpha  \
Segment Pollutant Fuel     Euro Standard                                       
BUS ART EC        Diesel   E1                            76.142857  0.007774   
                           E2                            76.857143  0.002040   
                           E3                            77.857143  0.007769   
                           E4                            77.142857 -0.005237   
                           E5                            77.142857  0.074894   
...                                                            ...       ...   
VLP     EC        Gasolina ECE 15/04                    130.000000  0.000107   
                           Euro 6 d-temp                130.000000  0.000132   
                           Improved Conventional        130.000000  2.862600   
                           No cat                       130.000000  0.000093   
                           Open Loop                    130.000000  0.053167   

                                                        Beta         Gamma  \
Segment Pollutant Fuel     Euro Standard                                     
BUS ART EC        Diesel   E1                      -0.020192      3.842632   
                           E2                       0.073482      5.942181   
                           E3                       0.234804      5.090711   
                           E4                       0.361103      1.374954   
                           E5                       3.143109      3.529938   
...                                                      ...           ...   
VLP     EC        Gasolina ECE 15/04               -0.020717      1.015806   
                           Euro 6 d-temp            0.005486      2.619195   
                           Improved Conventional -341.996509  18691.094291   
                           No cat                  -0.009706      0.356131   
                           Open Loop               -6.266148    346.853453   

                                                         Delta       Epsilon  \
Segment Pollutant Fuel     Euro Standard                                       
BUS ART EC  

In [390]:
#Toques finales a CH4, PM exhaust
Archivos = ['CH4_reducido.xlsx','PM_reducido.xlsx']
for file in Archivos:
  df = pd.read_excel(file)
  df = df.loc[
              df['Mode'].isnull() |
              (df['Mode'] == 'Highway') |
              (df['Mode'] == 'Urban Off Peak')
              ]
  df.loc[df['Mode'] == 'Highway', 'Mode'] = 'Interurbano'
  df.loc[df['Mode'] == 'Urban Off Peak', 'Mode'] = 'Urbano'
  df.to_excel(file,index = False)